In [78]:
from html.parser import HTMLParser  
from urllib.parse import urljoin, urlparse        
from urllib.request import urlopen
import re
import pickle
import numpy as np

In [71]:
def clean(url):
    if url[0:4] != "http":
        url = "http://" + url

    idx = url.find('#')
    if idx != -1:
        url = url[:idx]
        
    l = len(url)
    if url[l - 1] == '/':
        url = url[:l - 1]

    return url


def get_domain(url):
    
    parsed_url = urlparse(url)
    return "{url.netloc}".format(url=parsed_url)


def valid(url, domain):
    
    if re.match(r'^https?://([\w-]*\.)?' + domain + r'.*$', url, re.M|re.I):
        return True
    else:
        return False


def contain_static(val):
    if re.match(r'^.*\.(jpg|jpeg|gif|png|css|js|ico|xml|rss|txt|exe|pdf).*$', val, re.M|re.I):
        return True
    else:
        return False

In [72]:
class HTMLParser(HTMLParser):
    
    def handle_starttag(self, tag, attrs):
        for key, val in attrs:
            if key == "href":
                if contain_static(val):             
                    continue                 
                elif tag == "a":                    
                    url = urljoin(self.url, val)    
                    url = clean(url)                
                    if valid(url, self.domain):
                        self.urls.append(url)       
                else:
                    pass


    def run(self, url):
        '''
        Run the parser and return links in this page
        '''
        self.url = url                  
        self.domain = get_domain(url)  
        self.urls = []                  
        global count
        
        try:
            response = urlopen(url)                 
            html = response.read().decode("utf-8") 
            f = open('webdocs/'+str(count)+'.html', 'w')
            f.write(html)
            f.close
            self.feed(html)                         
        except KeyboardInterrupt:                   
            exit()
        except:
            print("Unexpected failure happens and the spider escapes.")

        return self.urls

In [73]:
class Crawler(object):
    def __init__(self):
        self.to_visit = []
        self.visted = set([])
        self.parser = HTMLParser()

    def crawl(self, target_url):
        global count
        target_url = clean(target_url)     
        self.to_visit.append(target_url)    
        reflist = []
        adjdict = {}

        while len(self.to_visit) > 0:
            url = self.to_visit.pop(0)      
            print("The Crawler is visiting:", url)
            urls = self.parser.run(url)     
            self.visted.add(url)
            reflist.append(url)
            adjdict[url] = set([])
            if(count==99):
                print("The Crawler has finished crawling the web at {url}".format(url=target_url))
                return reflist , adjdict
            count+=1

            
            
            for ur in urls:
                adjdict[url].add(ur)
                if ur not in self.visted and ur not in self.to_visit:
                    self.to_visit.append(ur)
        
                    

In [74]:
webcraw = Crawler()

In [75]:
count = 0
seedpage = "https://en.wikipedia.org/wiki/Cricket"

In [76]:
reflist , adjdict = webcraw.crawl(seedpage)

The Crawler is visiting: https://en.wikipedia.org/wiki/Cricket
The Crawler is visiting: https://en.wikipedia.org/wiki/Wikipedia:Protection_policy
The Crawler is visiting: https://en.wikipedia.org/wiki/Cricket_(insect)
The Crawler is visiting: https://en.wikipedia.org/wiki/Cricket_(disambiguation)
The Crawler is visiting: https://en.wikipedia.org/wiki/Cricketer_(disambiguation)
The Crawler is visiting: https://en.wikipedia.org/wiki/Eden_Gardens
The Crawler is visiting: https://en.wikipedia.org/wiki/2016_ICC_World_Twenty20_Final
The Crawler is visiting: https://en.wikipedia.org/wiki/Sports_governing_body
The Crawler is visiting: https://en.wikipedia.org/wiki/International_Cricket_Council
The Crawler is visiting: https://en.wikipedia.org/wiki/Contact_sport
The Crawler is visiting: https://en.wikipedia.org/wiki/Substitute_(cricket)
The Crawler is visiting: https://en.wikipedia.org/wiki/Mixed-sex_sports
The Crawler is visiting: https://en.wikipedia.org/wiki/Team_sport
The Crawler is visitin

In [83]:
size = len(reflist)

In [100]:
adjmat  = np.zeros((size,size)) 

In [101]:
for i, rt in enumerate(reflist):
    for j, out in enumerate(reflist):
        if out in adjdict[rt]:
            adjmat[i][j] = 1

In [102]:
print("saving web page reference list\n")
with open( "referencelist" + '.pkl', 'wb') as f:
        pickle.dump(reflist, f, pickle.HIGHEST_PROTOCOL)

saving web page reference list



In [103]:
print("saving adjacency matrix\n")
with open( "adjacencymatrix" + '.pkl', 'wb') as f:
        pickle.dump(adjmat, f, pickle.HIGHEST_PROTOCOL)

saving adjacency matrix

